# GECCO2019 - Bi-objective Traveling Thief Problem 

This worksheet contains the evaluation for the competition at **GECCO2019**. 
After having received all submissions, the evaluation will be done as follows:

After having received all submissions, the evaluation will be done as follows:

For each of the nine test problems

a) We will merge the solution sets of all submissions and extract the non-dominated set.

b) The minimum in time and the maximum in profit will be used to determine the reference point.

c) With respect to this reference point the quality of each submission will be measured using the hypervolume indicator.

d) We will sort the submissions according to the achieved hypervolume in descending order and give points as follows: 1st place -> 3 points, 2nd place -> 2 points, 3rd place -> 1 point.


By adding up the points for each submission we will create the overall ranking. Please note, that depending on the number of submissions the evaluation might need to be reconsidered.

The validation has already been done using the Java code. Which means that each submission has the correct number of solutions (less than the maximum specfied at the competition homepage).


## Imports necessary for the evaluation

In [1]:
from non_dominated_sorting import fast_non_dominated_sort
import os.path
import numpy as np
import matplotlib.pyplot as plt
from hv import Hypervolume
from normalization import normalize
import pandas as pd

## Participants and Problems

In [2]:
# the result folder as a path
folder = os.path.join(os.path.dirname(os.getcwd()), "submissions")


# all submissions received
participants = [
    "TeamU",
    "ALLAOUI","jomar", "shisunzhang",  "faria", "HPI",
    "NTGA", "SSteam", "SamirO-ETF-ba", "FRA", "sinc", "JG",
    # "ValAurTeam", "MicroGA" ## unfortunatly those submissions were invalid  
]


# all the problems to be solved
problems = ["a280_n279", "a280_n1395", "a280_n2790",
            "fnl4461_n4460", "fnl4461_n22300", "fnl4461_n44600", 
            "pla33810_n33809", "pla33810_n169045", "pla33810_n338090"
]


## Load data

Load all data from the submission directory and convert it to a minimization problem.
The data dictionary contains all submissions of a participant. The set of non-dominated points
is converted to a minimization problem by multiplying the profit times -1.

In [3]:

data = {}

for problem in problems:
    _entry = {}
    for participant in participants:
        
        # check for the corresponding file
        fname = "%s_%s.f" % (participant, problem)   
        path_to_file = os.path.join(folder,participant, fname)
        
        # in case the wrong delimiter was used
        if not os.path.isfile(path_to_file):
            fname = "%s_%s.f" % (participant, problem.replace("_", "-"))
            path_to_file = os.path.join(folder,participant, fname)

        if not os.path.isfile(path_to_file):
            print(path_to_file)
            break
         
        # load the values in the objective space - first column is time, second profit
        _F = np.loadtxt(path_to_file)
        
        # modify it to a min-min problem by multiplying the profit by -1
        _entry[participant] = _F * [1, -1]
        
    data[problem] = _entry




/Users/dathd6/Developments/NIC/GECCO2019---Bi-objective-Traveling-Thief-Competition/submissions/TeamUALLAOUI/TeamUALLAOUI_a280-n279.f
/Users/dathd6/Developments/NIC/GECCO2019---Bi-objective-Traveling-Thief-Competition/submissions/TeamUALLAOUI/TeamUALLAOUI_a280-n1395.f
/Users/dathd6/Developments/NIC/GECCO2019---Bi-objective-Traveling-Thief-Competition/submissions/TeamUALLAOUI/TeamUALLAOUI_a280-n2790.f
/Users/dathd6/Developments/NIC/GECCO2019---Bi-objective-Traveling-Thief-Competition/submissions/TeamUALLAOUI/TeamUALLAOUI_fnl4461-n4460.f
/Users/dathd6/Developments/NIC/GECCO2019---Bi-objective-Traveling-Thief-Competition/submissions/TeamUALLAOUI/TeamUALLAOUI_fnl4461-n22300.f
/Users/dathd6/Developments/NIC/GECCO2019---Bi-objective-Traveling-Thief-Competition/submissions/TeamUALLAOUI/TeamUALLAOUI_fnl4461-n44600.f
/Users/dathd6/Developments/NIC/GECCO2019---Bi-objective-Traveling-Thief-Competition/submissions/TeamUALLAOUI/TeamUALLAOUI_pla33810-n33809.f
/Users/dathd6/Developments/NIC/GECCO2019

## Plot the results

To get an idea how the submissions look like, we are plotting the results first.
Note that the plots are not normalized yet and the ranges of both object vary.

In [ ]:
print("Plot the results. If points are not shown there are not in the non-dominated region.")

import matplotlib.cm as cm
cmap = cm.get_cmap('tab20')

only_top_3 = False

for problem in problems:
    for k, participant in enumerate(participants):
        
        if not only_top_3 or (only_top_3 and participant in ["HPI", "jomar", "TeamU"]):
            if participant in data[problem]:
                _F = data[problem][participant]
                plt.scatter(_F[:,0], _F[:,1], label=participant, s=10, facecolors='none', edgecolors=cmap(k))

    if not data[problem]:
        continue
    #_all = np.row_stack([data[problem][participant] for participant in participants if participant in data[problem]])
    #I = fast_non_dominated_sort(_all)[0]
    #_non_dom = _all[I]
    
    #_min = _non_dom.min(axis=0)
    #_max = _non_dom.max(axis=0)
    #_range = _max - _min
        
    print("=" * 60)
    print(problem)
    print("=" * 60)
    plt.xlabel("time")
    plt.ylabel("negative profit")
    #plt.xlim(_min[0] - 0.05 * _range[0], _max[0] + 0.05 * _range[0])
    #plt.ylim(_min[1] - 0.05 * _range[1], _max[1] + 0.05 * _range[1])
    plt.legend()
    plt.show()
        

Plot the results. If points are not shown there are not in the non-dominated region.


/var/folders/mf/x389nv9s74n7bkxsm4l853k40000gn/T/ipykernel_73192/2585453793.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab20')


## Find the reference non-dominated set for each test instance

For each problem we merge the submissions to a new population and filter out the non-dominated solutions. Then, we take the minimum and the maximum of this set as the ideal and nadir point the normalize the results.

In [ ]:

ideal_point = {}
nadir_point = {}
ndf = {}

for problem in problems:
    
    # the merged non-dominated solutions for the specific problem
    M = []
    if not data[problem]:
        continue
    for participant in participants:    
        if participant in data[problem]:
            _F = data[problem][participant]
            M.append(_F)
        
    M = np.vstack(M)    
    I = fast_non_dominated_sort(M)[0]
    M = M[I, :]
    
    ideal_point[problem] = np.min(M, axis=0)
    nadir_point[problem] = np.max(M, axis=0)
    ndf[problem] = M
    
    

In the following for each problem the non-dominated set of solutions is first normalized using the boundaries and hypervolume is calculated.

In [ ]:
results = []

for problem in problems:
    if not data[problem]:
        continue
    
    z = ideal_point[problem]
    z_nad = nadir_point[problem]
    _N = np.array(normalize(data[problem]['TeamU'], z, z_nad))
    max_obj1 = _N[:, 0].max()
    max_obj2 = _N[:, 1].max()
 
    for participant in participants:    
        if participant in data[problem]:
            _F = data[problem][participant]
            _N = normalize(_F, z, z_nad)
            if problem == 'a280_n2790' and participant == 'TeamU':
                print(_N)
            _hv = Hypervolume(np.array([max_obj1, max_obj2])).calc(_N)
            results.append({'problem' : problem, 'participant' : participant, 'hv' : _hv})
        
df = pd.DataFrame(results, columns=["problem", "participant", "hv"])


In [ ]:
for problem in problems:

    print("=" * 60)
    print(problem)
    print("=" * 60)
    
    _df = df[df["problem"] == problem].copy()
    _df.sort_values("hv", ascending=False, inplace=True)
    _df.reset_index(drop=True, inplace=True)
    print(_df)

a280_n279
Empty DataFrame
Columns: [problem, participant, hv]
Index: []
a280_n1395
Empty DataFrame
Columns: [problem, participant, hv]
Index: []
a280_n2790
Empty DataFrame
Columns: [problem, participant, hv]
Index: []
fnl4461_n4460
Empty DataFrame
Columns: [problem, participant, hv]
Index: []
fnl4461_n22300
Empty DataFrame
Columns: [problem, participant, hv]
Index: []
fnl4461_n44600
Empty DataFrame
Columns: [problem, participant, hv]
Index: []
pla33810_n33809
Empty DataFrame
Columns: [problem, participant, hv]
Index: []
pla33810_n169045
Empty DataFrame
Columns: [problem, participant, hv]
Index: []
pla33810_n338090
Empty DataFrame
Columns: [problem, participant, hv]
Index: []


In [5]:
df[df['problem'] == 'a280_n2790']

NameError: name 'df' is not defined

The data frame contains all results. Now, we need to rank the submission for each test instance:

In [ ]:
# the final ranking. And add zero points initially (sum is later taken anyway...)
ranking = []
for participant in participants:
    ranking.append({'participant': participant, 'points' : 0})


# one more time loop through problem wise
for problem in problems:
    
    try:
        _df = df[df["problem"] == problem].copy()
        
        # sort descending by hv
        _df.sort_values("hv", ascending=False, inplace=True)
        
        # 3 points for the 1st place
        first = _df.iloc[0]["participant"]
        ranking.append({'participant': first, 'points' : 3})
        
        # 2 points for the 2nd place
        second = _df.iloc[1]["participant"]
        ranking.append({'participant': second, 'points' : 2})
        
        # 1 point for the 3rd place
        third = _df.iloc[2]["participant"]
        ranking.append({'participant': third, 'points' : 1})
    except:
        pass

    
ranking = pd.DataFrame(ranking, columns=["participant", "points"])

# Leaderboard

Finally, we sum up the hypervolume for each problem and evaluate the winner!

In [ ]:
ranking.groupby('participant').sum().sort_values("points", ascending=False)

,points
participant,
TeamU,6
HPI,5
jomar,5
ALLAOUI,1
shisunzhang,1
FRA,0
JG,0
NTGA,0
SSteam,0


## Leaderboard as HTML

In [ ]:


standalone = True
if standalone:
    print("<script src='https://code.jquery.com/jquery-3.2.1.slim.min.js' integrity='sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN' crossorigin='anonymous'></script>")
    print("<script src='https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js' integrity='sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q' crossorigin='anonymous'></script>")
    print("<script src='https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js' integrity='sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl' crossorigin='anonymous'></script>")

print("<div class='row'><div class='w-50'>")
print("<table class='table'>")
print("<tr><th>Problem</th><th>Team</th><th>Hypervolume</th><th>Estimated Ideal Point</th><th>Estimated Nadir Point</th></tr>")

for problem in problems:
    
    _df = df[df["problem"] == problem].copy()
    _df.sort_values("hv", ascending=False, inplace=True)
    _df.reset_index(drop=True)
    
    first = True
    for i, e in _df.iterrows():
        
        problem, team, hv = e.values
        
        if first:
            
            s_row_span = "<th rowspan='%s'>%s</th>"
            
            s_problem = s_row_span % (len(_df), problem)
            s_ideal = s_row_span % (len(_df), "(%s, %s)" % (round(ideal_point[problem][0]), round(ideal_point[problem][1])))
            s_nadir = s_row_span % (len(_df), "(%s, %s)" % (round(nadir_point[problem][0]), round(nadir_point[problem][1])))
            
            print("<tr>%s <th>%s</th> <th>%s</th> %s %s  </tr>" 
                  % (s_problem, team, round(hv,4), s_ideal, s_nadir))
        else:
            print("<tr><th>%s</th><th>%s</th></tr>" % (team, round(hv,4)))
        
            
        first = False
        
    print()
    
    
print("<table>")
print("</div></div>")

<script src='https://code.jquery.com/jquery-3.2.1.slim.min.js' integrity='sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN' crossorigin='anonymous'></script>
<script src='https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js' integrity='sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q' crossorigin='anonymous'></script>
<script src='https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js' integrity='sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl' crossorigin='anonymous'></script>
<div class='row'><div class='w-50'>
<table class='table'>
<tr><th>Problem</th><th>Team</th><th>Hypervolume</th><th>Estimated Ideal Point</th><th>Estimated Nadir Point</th></tr>
<tr><th rowspan='13'>a280_n279</th> <th>HPI</th> <th>0.8984</th> <th rowspan='13'>(2613, -42036)</th> <th rowspan='13'>(5444, 0)</th>  </tr>
<tr><th>jomar</th><th>0.8956</th></tr>
<tr><th>shisunzhang</th><th>0.8866</th></tr>
<tr><th>NTG

In [ ]:
_ranking = ranking.groupby('participant').sum().sort_values("points", ascending=False)

standalone = True
if standalone:
    print("<script src='https://code.jquery.com/jquery-3.2.1.slim.min.js' integrity='sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN' crossorigin='anonymous'></script>")
    print("<script src='https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js' integrity='sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q' crossorigin='anonymous'></script>")
    print("<script src='https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js' integrity='sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl' crossorigin='anonymous'></script>")


print("<div class='row'><div class='w-50'>")
print("<table class='table'>")
print("<tr><th>Rank</th><th>Team</th><th>Points</th></tr>")


counter = 0
for i, e in _ranking.iterrows():
    print("<tr><th>%s</th><th>%s</th><th>%s</th></tr>" % (counter+1, i, e[0]))
    counter += 1

    
print("<table>")
print("</div></div>")


<script src='https://code.jquery.com/jquery-3.2.1.slim.min.js' integrity='sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN' crossorigin='anonymous'></script>
<script src='https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js' integrity='sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q' crossorigin='anonymous'></script>
<script src='https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js' integrity='sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl' crossorigin='anonymous'></script>
<div class='row'><div class='w-50'>
<table class='table'>
<tr><th>Rank</th><th>Team</th><th>Points</th></tr>
<tr><th>1</th><th>TeamU</th><th>6</th></tr>
<tr><th>2</th><th>HPI</th><th>5</th></tr>
<tr><th>3</th><th>jomar</th><th>5</th></tr>
<tr><th>4</th><th>ALLAOUI</th><th>1</th></tr>
<tr><th>5</th><th>shisunzhang</th><th>1</th></tr>
<tr><th>6</th><th>FRA</th><th>0</th></tr>
<tr><th>7</th><th>JG</th><th>0</th><

/var/folders/mf/x389nv9s74n7bkxsm4l853k40000gn/T/ipykernel_73192/3062425893.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("<tr><th>%s</th><th>%s</th><th>%s</th></tr>" % (counter+1, i, e[0]))
